In [19]:

from collections import OrderedDict
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)
import torchvision
from torchvision.transforms import transforms

class Autoencoder(nn.Module):
    """
    Autoencoder class, combines encoder and decoder model.
    """
    
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = ConvNetEncoder()
        self.decoder = ConvNetDecoder()
    
    @property
    def num_params(self):
        model_parameters = filter(lambda p: p.requires_grad, self.parameters())
        num_p = sum([np.prod(p.size()) for p in model_parameters])
        return num_p
    
    def forward(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded ,encoded

class ConvNetEncoder(nn.Module):
    def __init__(self):
        super(ConvNetEncoder, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1), #1 * 96 * 192 -> 16 * 96 * 192
            nn.BatchNorm2d(64), #16 * 96 * 192
            nn.LeakyReLU(),                 
            nn.MaxPool2d(2),
            nn.Dropout(0.25))       # 16 * 48 * 96
        self.layer2  = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1), # 16* 48 * 96 -> 32 * 48 * 96
            nn.BatchNorm2d(128), #  32 * 48 * 96
            nn.ReLU(), #  32 * 48 * 96
            # nn.MaxPool2d(2),
            nn.Dropout(0.25))       # 32 * 24 * 48
        self.layer3  = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1), # 32 * 24 * 48 -> 64 * 24 * 48 
            nn.BatchNorm2d(256), #  64 * 24 * 48 
            nn.LeakyReLU(),                
            nn.MaxPool2d(2),
            nn.Dropout(0.25))       # 64 * 12 * 24
        self.layer4  = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, padding=1), # 64 * 12 * 24 -> 128 * 12 * 24
            nn.BatchNorm2d(512), #  128 * 12 * 24 
            nn.LeakyReLU(),                 
            nn.MaxPool2d(2),
            nn.Dropout(0.25))
        self.linlayer = nn.Sequential(
            nn.Linear(512*6*12, 1024),
            nn.LeakyReLU(),                 
            nn.Dropout(0.25),
            nn.Linear(1024, 16),
            nn.LeakyReLU(),                 
            nn.Dropout(0.5)
        )
        self.flatten = nn.Flatten()
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.flatten(out)
        out = self.linlayer(out)
        return out

class ConvNetDecoder(nn.Module):
    def __init__(self):
        super(ConvNetDecoder, self).__init__()    
        self.layer1 = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=0), 
            nn.BatchNorm2d(256),
            nn.ReLU(),            
            nn.Dropout(0.25))      
        self.layer2  = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=2, stride=1, padding=1), 
            nn.BatchNorm2d(128), 
            nn.ReLU(),
            nn.Dropout(0.25))      
        self.layer3  = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64), 
            nn.ReLU(),
            nn.Dropout(0.25))       
        self.layer4  = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=2, stride=1, padding=0),
            nn.BatchNorm2d(32), 
            nn.ReLU(),
            nn.Dropout(0.25))  
        self.tanh = nn.Tanh() 

        self.linlayer = nn.Sequential(
            nn.Linear(16, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512*6*12),
            nn.ReLU(),
            nn.Dropout(0.25)
        )
        
    def forward(self, x):
        out = self.linlayer(x)
        out = out.view(-1, 512, 6, 12)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.tanh(out)
        return out

In [20]:
x = torch.randn(1,1, 96,192)
autoencoder = Autoencoder()
decoder, encoder = autoencoder(x)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x589824 and 36864x1024)

In [ ]:
print("Encoder", encoder.shape)
print("Decoder", decoder.shape)

Encoder torch.Size([1, 16])
Decoder torch.Size([1, 1, 96, 192])


In [12]:
autoencoder.num_params

82170931